In [1]:
import sys
relative_root = "../.."
sys.path.append(relative_root)

In [2]:
import joblib
import rich
import pandas as pd
from sklearn.metrics import accuracy_score
import json
from examples.training import titanic_config
from pathlib import Path

## Load data & model

In [3]:
try:
    preprocessed_train = pd.read_csv(Path(relative_root) / titanic_config.LOCAL_TRAIN_FILENAME)
    preprocessed_test = pd.read_csv(Path(relative_root) / titanic_config.LOCAL_TEST_FILENAME)
    rf_model = joblib.load(Path(relative_root) / titanic_config.LOCAL_MODEL_FILENAME)
    with open(Path(relative_root) / titanic_config.LOCAL_FI_FILENAME, "r") as file:
        feature_importance = json.loads(file.read())
except FileNotFoundError:
    raise FileNotFoundError("To generate these files, run `make train-titanic`")

In [4]:
rf_model.predict(preprocessed_test)[0]  # test model inference

1

## Init trubrics context

In [5]:
from trubrics.context import DataContext

In [6]:
data_context = DataContext(
    name="my_first_dataset",
    version=0.1,
    training_data=preprocessed_train,
    testing_data=preprocessed_test,
    target_column=titanic_config.TARGET
)

## Init trubrics validator

In [7]:
from trubrics.validations.base import Validator

In [8]:
import sklearn.metrics as s
rich.print("Choose error metric from sklearn defaults: ", list(s.SCORERS))

Choose error metric from sklearn defaults: 
[
    'explained_variance',
    'r2',
    'max_error',
    'neg_median_absolute_error',
    'neg_mean_absolute_error',
    'neg_mean_absolute_percentage_error',
    'neg_mean_squared_error',
    'neg_mean_squared_log_error',
    'neg_root_mean_squared_error',
    'neg_mean_poisson_deviance',
    'neg_mean_gamma_deviance',
    'accuracy',
    'top_k_accuracy',
    'roc_auc',
    'roc_auc_ovr',
    'roc_auc_ovo',
    'roc_auc_ovr_weighted',
    'roc_auc_ovo_weighted',
    'balanced_accuracy',
    'average_precision',
    'neg_log_loss',
    'neg_brier_score',
    'adjusted_rand_score',
    'rand_score',
    'homogeneity_score',
    'completeness_score',
    'v_measure_score',
    'mutual_info_score',
    'adjusted_mutual_info_score',
    'normalized_mutual_info_score',
    'fowlkes_mallows_score',
    'precision',
    'precision_macro',
    'precision_micro',
    'precision_samples',
    'precision_weighted',
    'recall',
    'recall_macro',
    'recall_micro',
    'recall_samples',
    'recall_weighted',
    'f1',
    'f1_macro',
    'f1_micro',
    'f1_samples',
    'f1_weighted',
    'jaccard',
    'jaccard_macro',
    'jaccard_micro',
    'jaccard_samples',
    'jaccard_weighted'
]

In [15]:
model_validator = Validator(metric="accuracy", data=data_context, model=rf_model)
rich.print(f"Test set {model_validator.metric} score", model_validator.score_test)

Scoring on testing set...


Test set accuracy score 0.7966101694915254

## Use the trubrics validator to create out-of-the-box validations

In [16]:
edge_case = {
    "Sex": "male",
    "Embarked": "S",
    "Title": "Master",
    "Pclass": 2,
    "Age": 28,
    "SibSp": 0,
    "Parch": 0,
    "Fare": 37
}

In [17]:
robustness = [
    model_validator.validate_single_edge_case(edge_case_data=edge_case, desired_output=0), # example of fail
    model_validator.validate_single_edge_case(edge_case_data=edge_case, desired_output=1) # example of pass
]
robustness[0].severity = "warning"  # change the severity of a validation
robustness[1].severity = "experiment"

_ = [rich.print(val.dict()) for val in robustness]

{
    'validation_type': 'validate_single_edge_case',
    'validation_kwargs': {
        'args': (),
        'kwargs': {
            'edge_case_data': {
                'Sex': 'male',
                'Embarked': 'S',
                'Title': 'Master',
                'Pclass': 2,
                'Age': 28,
                'SibSp': 0,
                'Parch': 0,
                'Fare': 37
            },
            'desired_output': 0
        }
    },
    'outcome': 'fail',
    'severity': 'warning',
    'result': {'prediction': 1}
}

{
    'validation_type': 'validate_single_edge_case',
    'validation_kwargs': {
        'args': (),
        'kwargs': {
            'edge_case_data': {
                'Sex': 'male',
                'Embarked': 'S',
                'Title': 'Master',
                'Pclass': 2,
                'Age': 28,
                'SibSp': 0,
                'Parch': 0,
                'Fare': 37
            },
            'desired_output': 1
        }
    },
    'outcome': 'pass',
    'severity': 'experiment',
    'result': {'prediction': 1}
}

In [18]:
performance = [
    model_validator.validate_performance_against_threshold(threshold=0.8),
    model_validator.validate_performance_against_threshold(threshold=0.75),
    model_validator.validate_performance_against_dummy(),
    model_validator.validate_performance_against_dummy(strategy="stratified"),
]
_ = [rich.print(val.dict()) for val in performance]

{
    'validation_type': 'validate_performance_against_threshold',
    'validation_kwargs': {'args': (), 'kwargs': {'threshold': 0.8}},
    'outcome': 'pass',
    'severity': 'error',
    'result': {'performance': 0.7966101694915254}
}

{
    'validation_type': 'validate_performance_against_threshold',
    'validation_kwargs': {'args': (), 'kwargs': {'threshold': 0.75}},
    'outcome': 'fail',
    'severity': 'error',
    'result': {'performance': 0.7966101694915254}
}

{
    'validation_type': 'validate_performance_against_dummy',
    'validation_kwargs': {'args': (), 'kwargs': {}},
    'outcome': 'pass',
    'severity': 'error',
    'result': {'dummy_performance': 0.6508474576271186, 'test_performance': 0.7966101694915254}
}

{
    'validation_type': 'validate_performance_against_dummy',
    'validation_kwargs': {'args': (), 'kwargs': {'strategy': 'stratified'}},
    'outcome': 'pass',
    'severity': 'error',
    'result': {'dummy_performance': 0.5084745762711864, 'test_performance': 0.7966101694915254}
}

In [19]:
fairness = [
    model_validator.validate_biased_performance_across_category(category="Embarked", threshold=0.1),
    model_validator.validate_biased_performance_across_category(category="Sex", threshold=0.05)
]
_ = [rich.print(val.dict()) for val in fairness]

{
    'validation_type': 'validate_biased_performance_across_category',
    'validation_kwargs': {'args': (), 'kwargs': {'category': 'Embarked', 'threshold': 0.1}},
    'outcome': 'fail',
    'severity': 'error',
    'result': {'max_performance_difference': 0.15235267245317496}
}

{
    'validation_type': 'validate_biased_performance_across_category',
    'validation_kwargs': {'args': (), 'kwargs': {'category': 'Sex', 'threshold': 0.05}},
    'outcome': 'pass',
    'severity': 'error',
    'result': {'max_performance_difference': 0.0361426429642322}
}

In [20]:
explainability = [
    model_validator.validate_feature_in_top_n_important_features(feature="Sex_female", feature_importance=feature_importance, top_n_features=3),
    model_validator.validate_feature_in_top_n_important_features(feature="Age", feature_importance=feature_importance, top_n_features=2)
]
_ = [rich.print(val.dict()) for val in explainability]

AttributeError: 'Validator' object has no attribute 'validate_feature_in_top_n_important_features'

## Collect user feedback from app

In [21]:
from trubrics.utils.loader import get_business_feedback_data
# read test data and run single outlier test
try:
    data = get_business_feedback_data(tracking=False)
    display(data)
except FileNotFoundError:
    print("Please generate feedback from the streamlit app in order to read it back here")

{'feedback_type': 'Other',
 'metadata': {'description': 'model seems to have more errors for passengers who paid less - is this normal?',
  'what_if_input': {'Sex': {'0': 'male'},
   'Embarked': {'0': 'S'},
   'Title': {'0': 'Master'},
   'Pclass': {'0': 2},
   'Age': {'0': 28},
   'SibSp': {'0': 0},
   'Parch': {'0': 0},
   'Fare': {'0': 37}}}}

--> **DS response: "It isn't normal, the model should not be more accurate for different groups of people. I'll add a test for this."**

## Create custom validations

In [ ]:
from trubrics.validations.validation_output import validation_output, validation_output_type


class CustomValidator(Validator):
    def __init__(self, data: DataContext, model: ModelContext):
        super().__init__(data, model)
        
    @validation_output
    def validate_performance_for_different_fares(self, fare_cutoff):
        return self._validate_performance_for_different_fares(fare_cutoff)

    def _validate_performance_for_different_fares(self, fare_cutoff: int = 50) -> validation_output_type:
        """
        Write your custom validation function here.
        
        Notes
        -----
            This method is separated from validate_performance_for_different_fares
            to apply @validation_output and for unit testing.

            The @validation_output decorator allows you to generate a Validation object,
            and must be used to be able to save your validation as part of a Trubric.
            This decorator requires you to return values with the same type as validation_output_type.
        """
        
        errors_df = self.trubrics_model.explore_test_set_errors()
        number_of_errors_by_fare_ratio = (
            errors_df.loc[lambda x: x["Fare"] <= fare_cutoff].shape[0]
            / errors_df.loc[lambda x: x["Fare"] > fare_cutoff].shape[0]
        )
        return (
            number_of_errors_by_fare_ratio > 0.5 and number_of_errors_by_fare_ratio < 1.5,
            {"number_of_errors_by_fare_ratio": round(number_of_errors_by_fare_ratio, 3)}
        )

In [ ]:
model_custom_validator = CustomValidator(data=data_context, model=model_context)

In [ ]:
custom = [model_custom_validator.validate_performance_for_different_fares(fare_cutoff=25)]
custom[0].severity = "warning"  # change the severity of a validation
_ = [rich.print(val.dict()) for val in custom]

## Save trubric

In [ ]:
from trubrics.context import TrubricContext

validations = robustness + performance + fairness + explainability + custom

trubric_context = TrubricContext(
    name="my_first_trubric",
    model_context_name=model_context.name,
    model_context_version=model_context.version,
    data_context_name=data_context.name,
    data_context_version=data_context.version,
    validations=validations,
)

trubric_context.save_local(path="../data")

In [ ]:
# or save to trubrics UI
user_id = None  # enter User ID from trubrics manager here
url = None  # enter api url for trubrics manager here

trubric_context = TrubricContext(
    name="my_first_trubric",
    model_context_name=model_context.name,
    model_context_version=model_context.version,
    data_context_name=data_context.name,
    data_context_version=data_context.version,
    metadata={"tag": "master"},
    validations=validations,
)


try:
    trubric_context.save_ui(url=url, user_id=user_id)
    print("Trubric saved to UI.")
except Exception as e:
    print(f"Error in saving to trubrics manager:\n{e}")

In [ ]:
trubric_context.dict()

## Execute trubric from file

In [ ]:
from trubrics.validations.run import run_trubric

In [ ]:
trubric = TrubricContext.parse_file("../data/my_first_trubric.json")

all_validation_results = run_trubric(
    data_context=data_context,
    model_context=model_context,
    custom_validator=model_custom_validator,
    trubric=trubric
)

In [ ]:
for validation_result in all_validation_results:
    rich.print(f"{validation_result.validation_type} - {validation_result.severity.upper()}.......{validation_result.outcome}")

In [ ]:
# or run from cli
import os
os.chdir("../..")
!make example-run-trubric 
os.chdir("examples/notebooks")